In [1]:
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dropout
import tensorflow as tf
from tqdm import tqdm
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer

%matplotlib inline

2024-01-24 12:44:03.879479: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-24 12:44:04.596670: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 12:44:04.596877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 12:44:04.676455: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 12:44:05.546588: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-24 12:44:05.555333: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
dict_2classes = {}
dict_2classes['DDoS-RSTFINFlood'] = 'Attack'
dict_2classes['DDoS-PSHACK_Flood'] = 'Attack'
dict_2classes['DDoS-SYN_Flood'] = 'Attack'
dict_2classes['DDoS-UDP_Flood'] = 'Attack'
dict_2classes['DDoS-TCP_Flood'] = 'Attack'
dict_2classes['DDoS-ICMP_Flood'] = 'Attack'
dict_2classes['DDoS-SynonymousIP_Flood'] = 'Attack'
dict_2classes['DDoS-ACK_Fragmentation'] = 'Attack'
dict_2classes['DDoS-UDP_Fragmentation'] = 'Attack'
dict_2classes['DDoS-ICMP_Fragmentation'] = 'Attack'
dict_2classes['DDoS-SlowLoris'] = 'Attack'
dict_2classes['DDoS-HTTP_Flood'] = 'Attack'

dict_2classes['DoS-UDP_Flood'] = 'Attack'
dict_2classes['DoS-SYN_Flood'] = 'Attack'
dict_2classes['DoS-TCP_Flood'] = 'Attack'
dict_2classes['DoS-HTTP_Flood'] = 'Attack'


dict_2classes['Mirai-greeth_flood'] = 'Attack'
dict_2classes['Mirai-greip_flood'] = 'Attack'
dict_2classes['Mirai-udpplain'] = 'Attack'

dict_2classes['Recon-PingSweep'] = 'Attack'
dict_2classes['Recon-OSScan'] = 'Attack'
dict_2classes['Recon-PortScan'] = 'Attack'
dict_2classes['VulnerabilityScan'] = 'Attack'
dict_2classes['Recon-HostDiscovery'] = 'Attack'

dict_2classes['DNS_Spoofing'] = 'Attack'
dict_2classes['MITM-ArpSpoofing'] = 'Attack'

dict_2classes['BenignTraffic'] = 'Benign'

dict_2classes['BrowserHijacking'] = 'Attack'
dict_2classes['Backdoor_Malware'] = 'Attack'
dict_2classes['XSS'] = 'Attack'
dict_2classes['Uploading_Attack'] = 'Attack'
dict_2classes['SqlInjection'] = 'Attack'
dict_2classes['CommandInjection'] = 'Attack'

dict_2classes['DictionaryBruteForce'] = 'Attack'

In [3]:

file_paths = glob.glob('/home/haohao/Downloads/archive/wataiData/csv/CICIoT2023/*.csv')
file_paths.sort()
print('file_paths len: ', len(file_paths))

df = pd.DataFrame()
#[:round(len(file_paths)*0.01)]
for file_path in tqdm(file_paths[:round(len(file_paths)*0.6)], desc='Processing files', unit='file'):
    dff = pd.read_csv(file_path)
    
    df = pd.concat([df, dff], ignore_index=True)

file_paths len:  169


Processing files:   0%|          | 0/118 [00:00<?, ?file/s]

Processing files: 100%|██████████| 118/118 [08:49<00:00,  4.49s/file]


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31625860 entries, 0 to 31625859
Data columns (total 47 columns):
 #   Column           Dtype  
---  ------           -----  
 0   flow_duration    float64
 1   Header_Length    float64
 2   Protocol Type    float64
 3   Duration         float64
 4   Rate             float64
 5   Srate            float64
 6   Drate            float64
 7   fin_flag_number  float64
 8   syn_flag_number  float64
 9   rst_flag_number  float64
 10  psh_flag_number  float64
 11  ack_flag_number  float64
 12  ece_flag_number  float64
 13  cwr_flag_number  float64
 14  ack_count        float64
 15  syn_count        float64
 16  fin_count        float64
 17  urg_count        float64
 18  rst_count        float64
 19  HTTP             float64
 20  HTTPS            float64
 21  DNS              float64
 22  Telnet           float64
 23  SMTP             float64
 24  SSH              float64
 25  IRC              float64
 26  TCP              float64
 27  UDP       

In [5]:
df['label'] = df['label'].map(dict_2classes)
df = df.sample(frac=1, random_state=42)

In [6]:
df.head(10)

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
29635440,0.000000,0.0,1.0,64.0,24.015849,24.015849,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.312804e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,Attack
31192878,0.276910,75.6,6.0,64.0,2.768034,2.768034,0.0,0.0,1.0,0.0,...,0.000000,54.0,8.336143e+07,9.5,10.392305,0.000000,0.000000,0.0,141.55,Attack
31388688,0.000000,0.0,1.0,64.0,50.053750,50.053750,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.312898e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,Attack
8644612,0.000000,0.0,1.0,64.0,14.532774,14.532774,0.0,0.0,0.0,0.0,...,0.000000,42.0,8.312870e+07,9.5,9.165151,0.000000,0.000000,0.0,141.55,Attack
26404300,0.469571,108.0,6.0,64.0,4.264098,4.264098,0.0,0.0,1.0,0.0,...,0.000000,54.0,8.336134e+07,9.5,10.392305,0.000000,0.000000,0.0,141.55,Attack
3523688,0.298195,39822.5,17.0,64.0,4602.926793,4602.926793,0.0,0.0,0.0,0.0,...,0.000000,50.0,8.310616e+07,9.5,10.000000,0.000000,0.000000,0.0,141.55,Attack
14159581,0.067067,25545.0,17.0,64.0,7853.120470,7853.120470,0.0,0.0,0.0,0.0,...,0.000000,50.0,8.349523e+07,9.5,10.000000,0.000000,0.000000,0.0,141.55,Attack
24967922,564.505680,11581.8,6.5,129.6,0.195625,0.195625,0.0,0.0,0.0,0.0,...,44.629771,91.0,7.058258e-02,5.5,14.151485,63.116027,2241.860343,0.9,38.50,Attack
8980140,0.000000,54.0,6.0,64.0,85.132419,85.132419,0.0,0.0,1.0,0.0,...,0.000000,54.0,8.308973e+07,9.5,10.392305,0.000000,0.000000,0.0,141.55,Attack
28208597,3.805843,3588535.0,17.0,64.0,1701.981964,1701.981964,0.0,0.0,0.0,0.0,...,0.000000,554.0,8.371153e+07,9.5,33.286634,0.000000,0.000000,0.0,141.55,Attack


In [7]:
y = pd.get_dummies(df['label']).iloc[:, ::-1].to_numpy()

y_labels = np.argmax(y, axis=1)

print(y[0:10])
print(df['label'].head(10))
y_labels[:10]

[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
29635440    Attack
31192878    Attack
31388688    Attack
8644612     Attack
26404300    Attack
3523688     Attack
14159581    Attack
24967922    Attack
8980140     Attack
28208597    Attack
Name: label, dtype: object


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [8]:

# Đếm số lượng mỗi nhãn
label_counts = np.bincount(y_labels)

# In ra số lượng mỗi nhãn
for i, count in enumerate(label_counts):
    print(f"Label {i}: {count} occurrences")

Label 0: 743916 occurrences
Label 1: 30881944 occurrences


In [9]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(method='ffill', inplace=True)
scaler = MinMaxScaler()
X = df.iloc[:, :-1]
X = scaler.fit_transform(X)

In [10]:
X[0:10]

array([[0.00000000e+00, 0.00000000e+00, 2.12765957e-02, 2.50980392e-01,
        2.86291231e-06, 2.86291231e-06, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 3.13448102e-03, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.95874021e-01,
        6.07142857e-01, 2.77555756e-17, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 5.76970443e-01],
       [1.05322382e-06, 7.63085420e-06, 1.27659574e-01, 2.50980392e-01,
        3.29975307e-07, 3.29975307e-07, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00,

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y_labels, test_size=0.2, random_state=75)

In [12]:

# Đếm số lượng mỗi nhãn
label_counts = np.bincount(y_test)
#label_counts = np.bincount(y_train)

# In ra số lượng mỗi nhãn
for i, count in enumerate(label_counts):
    print(f"Label {i}: {count} occurrences")

Label 0: 148963 occurrences
Label 1: 6176209 occurrences


In [13]:
from sklearn.metrics import classification_report, confusion_matrix

def eval(model):
    # Dự đoán nhãn trên tập kiểm thử
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    y_pred = tf.squeeze(y_pred, axis=1)
    print(y_pred[:10])
    print(y_test[:10])
    cm = confusion_matrix(y_test, y_pred)
    # Tính toán độ chính xác tổng thể
    total_accuracy = (cm[0, 0] + cm[1, 1]) / np.sum(cm)


    # Tính toán F1-score, Recall và Precision
    report = classification_report(y_test, y_pred, target_names=None)

    # Thêm vào báo cáo
    report += '\nTotal accuracy: {:0.2f} \n'.format(total_accuracy)

    print(report)


In [14]:
def build_model():
  n_features = X_train.shape[1]
  model = Sequential()
  model.add(Dense(512, input_dim=n_features, activation='relu'))  # Input layer và hidden layer đầu tiên
  model.add(Dropout(0.3))  # Thêm dropout để giảm overfitting
  model.add(Dense(256, activation='relu'))  # Thêm hidden layer thứ hai
  model.add(Dropout(0.3))  # Thêm dropout
  model.add(Dense(128, activation='relu'))  # Thêm hidden layer thứ hai
  model.add(Dropout(0.3))  # Thêm dropout
  model.add(Dense(64, activation='relu'))  # Thêm hidden layer thứ hai
  model.add(Dropout(0.3))  # Thêm dropout
  model.add(Dense(1, activation='sigmoid'))  # Output layer

  # Compile mô hình
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = build_model()

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

eval(model)


: 

In [ ]:
eval(model)

2860/2860 [==============================] - 5s 2ms/step
tf.Tensor([1 1 1 1 1 1 1 1 1 1], shape=(10,), dtype=int64)
[1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.80      0.87      0.83      2153
           1       1.00      0.99      1.00     89346

    accuracy                           0.99     91499
   macro avg       0.90      0.93      0.91     91499
weighted avg       0.99      0.99      0.99     91499

Total accuracy: 0.99 



In [ ]:
# Tính tổng số mẫu
n_samples = X_train.shape[0]

# Tính chỉ số cho việc chia tách
# Phần 1: 0 -> 5/10 * n_samples
# Phần 2: 5/10 * n_samples -> (5+3)/10 * n_samples
# Phần 3: (5+3)/10 * n_samples -> n_samples
indices = [int(5/10 * n_samples), int((5+3)/10 * n_samples)]

# Chia dữ liệu
client_data = np.split(X_train, indices)
client_labels = np.split(y_train, indices)

# client_data[0], client_labels[0] sẽ chứa 50% dữ liệu đầu tiên
# client_data[1], client_labels[1] sẽ chứa 30% dữ liệu tiếp theo
# client_data[2], client_labels[2] sẽ chứa 20% dữ liệu cuối cùng


In [ ]:
def fed_avg(models, data_distribution):
    """
    models: danh sách các mô hình từ các node
    data_distribution: danh sách chứa số lượng dữ liệu mỗi node có
    """
    # Tính tổng số mẫu dữ liệu
    total_data = sum(data_distribution)

    # Khởi tạo trọng số trung bình
    avg_weights = [np.zeros_like(w) for w in models[0].get_weights()]

    # Cộng trọng số từ tất cả mô hình, đã được nhân với tỷ lệ dữ liệu tương ứng
    for model, data_count in zip(models, data_distribution):
        for idx, layer_weights in enumerate(model.get_weights()):
            avg_weights[idx] += (layer_weights * data_count) / total_data

    return avg_weights


In [ ]:
global_model = build_model()

In [ ]:
# Các thông số DP
l2_norm_clip = 1.0
noise_multiplier = 1.1
num_microbatches = 1  # Có thể cần phải điều chỉnh


# FL
num_rounds = 21
num_epochs = 3
batch_size = 128

In [ ]:

for round in range(num_rounds):
    local_models = []
    cnt = 1
    for client_X, client_y in zip(client_data, client_labels):
        print('Round ', round, ' - Client ', cnt)
        cnt += 1
        # Sao chép trọng số từ global model
        local_model = build_model()
        local_model.set_weights(global_model.get_weights())
        
        # Biên dịch mô hình với bộ tối ưu hóa DP
        local_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


        # Huấn luyện mô hình cục bộ
        local_model.fit(client_X, client_y, epochs=num_epochs, batch_size=batch_size, verbose=0)
        local_models.append(local_model)

    # Cập nhật global model dựa trên trung bình trọng số từ các mô hình cục bộ
    global_model.set_weights(fed_avg(local_models, [len(cli_dt) for cli_dt in client_data]))

    print('Round ', round, ' - Evaluation:')
    eval(global_model)

Round  0  - Client  1
Round  0  - Client  2
Round  0  - Client  3
Round  0  - Evaluation:
2860/2860 [==============================] - 5s 2ms/step
tf.Tensor([1 1 1 1 1 1 1 1 1 1], shape=(10,), dtype=int64)
[1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.66      0.99      0.79      2153
           1       1.00      0.99      0.99     89346

    accuracy                           0.99     91499
   macro avg       0.83      0.99      0.89     91499
weighted avg       0.99      0.99      0.99     91499

Total accuracy: 0.99 

Round  1  - Client  1
Round  1  - Client  2
Round  1  - Client  3


KeyboardInterrupt: 

In [ ]:
global_model = build_model()


for round in range(num_rounds):
    local_models = []
    cnt = 1
    for client_X, client_y in zip(client_data, client_labels):
        print('Round ', round, ' - Client ', cnt)
        cnt += 1
        # Sao chép trọng số từ global model
        local_model = build_model()
        local_model.set_weights(global_model.get_weights())
        
        optimizer = DPKerasSGDOptimizer(
            l2_norm_clip=l2_norm_clip,
            noise_multiplier=noise_multiplier,
            num_microbatches=num_microbatches,
            learning_rate=0.001
        )

        # Biên dịch mô hình với bộ tối ưu hóa DP
        local_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


        # Huấn luyện mô hình cục bộ
        local_model.fit(client_X, client_y, epochs=num_epochs, batch_size=batch_size, verbose=0)
        local_models.append(local_model)

    # Cập nhật global model dựa trên trung bình trọng số từ các mô hình cục bộ
    global_model.set_weights(fed_avg(local_models, [len(cli_dt) for cli_dt in client_data]))

    print('Round ', round, ' - Evaluation:')
    eval(global_model)

Round  0  - Client  1
Round  0  - Client  2
Round  0  - Client  3
Round  0  - Evaluation:
2860/2860 [==============================] - 5s 2ms/step
tf.Tensor([1 0 0 1 0 1 1 1 1 1], shape=(10,), dtype=int64)
[1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.01      0.13      0.02      2153
           1       0.97      0.76      0.85     89346

    accuracy                           0.75     91499
   macro avg       0.49      0.44      0.44     91499
weighted avg       0.95      0.75      0.83     91499

Total accuracy: 0.75 

Round  1  - Client  1
Round  1  - Client  2
Round  1  - Client  3
Round  1  - Evaluation:
2860/2860 [==============================] - 5s 2ms/step
tf.Tensor([1 0 0 1 0 1 1 1 1 1], shape=(10,), dtype=int64)
[1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.01      0.13      0.02      2153
           1       0.97      0.76      0.85     89346

    accuracy                  